## 3.4-抓取猫眼电影排行


本节中，我们利用requests库和正则表达式来抓取猫眼电影TOP100的相关内容。requests比urllib使用更加方便，而且目前我们还没有系统学习HTML解析库，所以这里就选用正则表达式来作为解析工具。


可以发现页面的URL变成http://maoyan.com/board/4?offset=10
，比之前的URL多了一个参数，那就是offset=10，而目前显示的结果是排行11~20名的电影，初步推断这是一个偏移量的参数。再点击下一页，发现页面的URL变成了http://maoyan.com/board/4?offset=20
，参数offset变成了20，而显示的结果是排行21~30的电影。

由此可以总结出规律，offset代表偏移量值，如果偏移量为n，则显示的电影序号就是n+1到n+10，每页显示10个。所以，如果想获取TOP100电影，只需要分开请求10次，而10次的offset参数分别设置为0、10、20、…90即可，这样获取不同的页面之后，再用正则表达式提取出相关信息，就可以得到TOP100的所有电影信息了。

4. 抓取首页

接下来用代码实现这个过程。首先抓取第一页的内容。我们实现了get_one_page()方法，并给它传入url参数。然后将抓取的页面结果返回，再通过main()方法调用。初步代码实现如下：


Q：猫眼会对自动化爬虫进行禁止，有时会被禁止访问
A:使用get.(url,headers = urlheaders)，将访问时的RequestHeaders添加进get参数


In [18]:
import requests

def get_one_page(url):
    MaoYanHeaders = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.8",
        "Connection": "keep-alive",
        "Cookie": "uuid=1A6E888B4A4B29B16FBA1299108DBE9CDFE0F270F2640051092C5B91D4925C7A; _lx_utm=utm_source%3Dbaidu%26utm_medium%3Dorganic; __mta=219052582.1507114997794.1507115797315.1507118482776.11; _lxsdk_s=af27c2388b4347ab08f2353fe7c8%7C%7C4",
        "Host": "maoyan.com",
        "Referer": "http://maoyan.com/board/4?offset=90/",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
    }
    response = requests.get(url,headers=MaoYanHeaders)
    print(response.status_code)
    if response.status_code == 200:
        return response.text
    return None

def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    print(html)

main()

200
<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//analytics.meituan.com" />
  <link rel="dns-prefetch" href="//report.meituan.com" />
  <link rel="dns-prefetch" href="//frep.meituan.com" />

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webkit" />

  <meta name="HandheldFriendly" content="tr

- 首先，需要提取它的排名信息。
- 随后需要提取电影的图片。
- 再往后，需要提取电影的名称，
- 再提取主演、发布时间、评分等内容时，

In [19]:
def parse_one_page(html):
    pattern = re.compile(
    '<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',re.S)
    items = re.findall(pattern, html)
    print(items)

In [22]:
import re

url = 'http://maoyan.com/board/4'
html = get_one_page(url)
parse_one_page(html)

200
[('1', 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '霸王别姬', '\n                主演：张国荣,张丰毅,巩俐\n        ', '上映时间：1993-01-01(中国香港)', '9.', '6'), ('2', 'http://p0.meituan.net/movie/__40191813__4767047.jpg@160w_220h_1e_1c', '肖申克的救赎', '\n                主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿\n        ', '上映时间：1994-10-14(美国)', '9.', '5'), ('3', 'http://p0.meituan.net/movie/23/6009725.jpg@160w_220h_1e_1c', '罗马假日', '\n                主演：格利高利·派克,奥黛丽·赫本,埃迪·艾伯特\n        ', '上映时间：1953-09-02(美国)', '9.', '1'), ('4', 'http://p0.meituan.net/movie/fc9d78dd2ce84d20e53b6d1ae2eea4fb1515304.jpg@160w_220h_1e_1c', '这个杀手不太冷', '\n                主演：让·雷诺,加里·奥德曼,娜塔莉·波特曼\n        ', '上映时间：1994-09-14(法国)', '9.', '5'), ('5', 'http://p0.meituan.net/movie/92/8212889.jpg@160w_220h_1e_1c', '教父', '\n                主演：马龙·白兰度,阿尔·帕西诺,詹姆斯·凯恩\n        ', '上映时间：1972-03-24(美国)', '9.', '3'), ('6', 'http://p0.meituan.net/movie/11/324629.jpg@160w_220h_1e_1c', '泰坦尼克号', '\n                主演：莱昂纳多·迪卡普

In [23]:
def parse_one_page(html):
    pattern = re.compile(
        '<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',
        re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield{
            'index':item[0],
            'image': item[1],
            'title': item[2].strip(),
            'actor': item[3].strip()[3:] if len(item[3]) > 3 else '',
            'time': item[4].strip()[5:] if len(item[4]) > 5 else '',
            'score': item[5].strip() + item[6].strip()
            
        }
        

In [25]:
import re

url = 'http://maoyan.com/board/4'
html = get_one_page(url)
parse_one_page(html)

200
<generator object parse_one_page at 0x000002A989C40678>


In [36]:

def write_to_file(content):
    with open('result.txt', 'a', encoding='utf-8') as f:
        print(type(json.dumps(content)))
        f.write(json.dumps(content, ensure_ascii=False) + '\n')        

In [37]:
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

In [38]:
import json
main()

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [39]:
def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
<class 'str'>
{'index': '2', 'image': 'http://p0.meituan.net/movie/__40191813__4767047.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
<class 'str'>
{'index': '3', 'image': 'http://p0.meituan.net/movie/23/6009725.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高利·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
<class 'str'>
{'index': '4', 'image': 'http://p0.meituan.net/movie/fc9d78dd2ce84d20e53b6d1ae2eea4fb1515304.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
<class 'str'>
{'index': '5', 'image': 'http://p0.meituan.net/movie/92/8212889.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·凯恩', 'time': '1972-03-24(美国)', 'score': '9

{'index': '41', 'image': 'http://p0.meituan.net/movie/9/1476994.jpg@160w_220h_1e_1c', 'title': '幽灵公主', 'actor': '松田洋治,石田百合子,田中裕子', 'time': '1997-07-12(日本)', 'score': '8.9'}
<class 'str'>
{'index': '42', 'image': 'http://p0.meituan.net/movie/54/868271.jpg@160w_220h_1e_1c', 'title': '阿凡达', 'actor': '萨姆·沃辛顿,佐伊·索尔达娜,米歇尔·罗德里格兹', 'time': '2010-01-04', 'score': '9.0'}
<class 'str'>
{'index': '43', 'image': 'http://p0.meituan.net/movie/57/5832292.jpg@160w_220h_1e_1c', 'title': 'V字仇杀队', 'actor': '娜塔莉·波特曼,雨果·维文,斯蒂芬·瑞', 'time': '2006-03-17(美国)', 'score': '8.8'}
<class 'str'>
{'index': '44', 'image': 'http://p0.meituan.net/movie/77/5392065.jpg@160w_220h_1e_1c', 'title': '风之谷', 'actor': '岛本须美,永井一郎,坂本千夏', 'time': '1992', 'score': '8.9'}
<class 'str'>
{'index': '45', 'image': 'http://p0.meituan.net/movie/7cd18fcf0b4f9180500124711e81492994030.jpg@160w_220h_1e_1c', 'title': '放牛班的春天', 'actor': '杰拉尔·朱诺,尚-巴堤·莫里耶,玛丽·布奈尔', 'time': '2004-10-16', 'score': '8.9'}
<class 'str'>
{'index': '46', 'image': 'http://

{'index': '81', 'image': 'http://p0.meituan.net/movie/9200962015fab182a516abf15629868d19268.jpg@160w_220h_1e_1c', 'title': '窃听风暴', 'actor': '乌尔里希·穆埃,赛巴斯汀‧柯赫,马蒂娜·戈黛特', 'time': '2006-03-23(德国)', 'score': '9.0'}
<class 'str'>
{'index': '82', 'image': 'http://p0.meituan.net/movie/7b3ec4254145fa618acfe02ba5090f4229614.jpg@160w_220h_1e_1c', 'title': '穿条纹睡衣的男孩', 'actor': '阿沙·巴特菲尔德,维拉·法梅加,大卫·休里斯', 'time': '2008-09-12(英国)', 'score': '9.0'}
<class 'str'>
{'index': '83', 'image': 'http://p0.meituan.net/movie/4abc8c932cfacfc0089e2883765d02d1295222.jpg@160w_220h_1e_1c', 'title': '时空恋旅人', 'actor': '瑞秋·麦克亚当斯,多姆纳尔·格里森,比尔·奈伊', 'time': '2013-09-04(英国)', 'score': '8.9'}
<class 'str'>
{'index': '84', 'image': 'http://p1.meituan.net/movie/1cb974ad0afb36582fc8090220417950344272.jpg@160w_220h_1e_1c', 'title': '地球上的星星', 'actor': '塔奈·切赫达,阿米尔·汗,Darsheel Safary', 'time': '2007-12-21(印度)', 'score': '8.8'}
<class 'str'>
{'index': '85', 'image': 'http://p1.meituan.net/movie/cf09a7f134015c6429b2329e929d7611150403.jp

In [35]:
import json
import requests
from requests.exceptions import RequestException
import re
import time

def get_one_page(url):
    MaoYanHeaders = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.8",
        "Connection": "keep-alive",
        "Cookie": "uuid=1A6E888B4A4B29B16FBA1299108DBE9CDFE0F270F2640051092C5B91D4925C7A; _lx_utm=utm_source%3Dbaidu%26utm_medium%3Dorganic; __mta=219052582.1507114997794.1507115797315.1507118482776.11; _lxsdk_s=af27c2388b4347ab08f2353fe7c8%7C%7C4",
        "Host": "maoyan.com",
        "Referer": "http://maoyan.com/board/4?offset=90/",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
    }
    
    try:
        response = requests.get(url,headers=MaoYanHeaders)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None    
    

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         + '.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         + '.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'index': item[0],
            'image': item[1],
            'title': item[2],
            'actor': item[3].strip()[3:],
            'time': item[4].strip()[5:],
            'score': item[5] + item[6]
        }

def write_to_file(content):
    # 'a' 继续往后面追加写 append
    with open('result.txt', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')

def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)
        time.sleep(1)

{'index': '1', 'image': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01(中国香港)', 'score': '9.6'}
{'index': '2', 'image': 'http://p0.meituan.net/movie/__40191813__4767047.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'http://p0.meituan.net/movie/23/6009725.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高利·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'http://p0.meituan.net/movie/fc9d78dd2ce84d20e53b6d1ae2eea4fb1515304.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'http://p0.meituan.net/movie/92/8212889.jpg@160w_220h_1e_1c', 'title': '教父', 'actor': '马龙·白兰度,阿尔·帕西诺,詹姆斯·凯恩', 'time': '1972-03-24(美国)', 'score': '9.3'}
{'index': '6', 'image': 'http://p0.meituan.net/movi

{'index': '51', 'image': 'http://p1.meituan.net/movie/f8e9d5a90224746d15dfdbd53d4fae3d209420.jpg@160w_220h_1e_1c', 'title': '勇敢的心', 'actor': '梅尔·吉布森,苏菲·玛索,帕特里克·麦高汉', 'time': '1995-05-24(美国)', 'score': '8.8'}
{'index': '52', 'image': 'http://p0.meituan.net/movie/44/43.jpg@160w_220h_1e_1c', 'title': '三傻大闹宝莱坞', 'actor': '阿米尔·汗,黄渤,卡琳娜·卡普', 'time': '2011-12-08', 'score': '9.1'}
{'index': '53', 'image': 'http://p0.meituan.net/movie/e71affe126eeb4f8bfcc738cbddeebc8288766.jpg@160w_220h_1e_1c', 'title': '断背山', 'actor': '希斯·莱杰,杰克·吉伦哈尔,米歇尔·威廉姆斯', 'time': '2006-01-13(美国)', 'score': '9.0'}
{'index': '54', 'image': 'http://p1.meituan.net/movie/01db9c3365a825ccd612aa14a12a36f528278.jpg@160w_220h_1e_1c', 'title': '神偷奶爸', 'actor': '史蒂夫·卡瑞尔,杰森·席格尔,拉塞尔·布兰德', 'time': '2010-07-09(美国)', 'score': '9.0'}
{'index': '55', 'image': 'http://p0.meituan.net/movie/62/499239.jpg@160w_220h_1e_1c', 'title': '飞屋环游记', 'actor': '爱德华·阿斯纳,乔丹·长井,鲍勃·彼德森', 'time': '2009-08-04', 'score': '8.9'}
{'index': '56', 'image': 'http://